In [ ]:
import requests
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)
cursor = conn.cursor()

url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/2/players"

headers = {
	"x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data = response.json()



In [ ]:
data.keys()
data.get("player", [])

In [ ]:
import requests
import pandas as pd
import mysql.connector
import time

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS team_players_updated (
    player_id INT PRIMARY KEY,
    team_id INT,
    player_name VARCHAR(100),
    batting_style VARCHAR(50),
    bowling_style VARCHAR(50),
    image_id INT,
    category VARCHAR(50)
)
""")
cursor.execute("SELECT team_id FROM international_teams")
team_ids = [row[0] for row in cursor.fetchall()]
for team_id in team_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}/players"
    response = requests.get(url, headers=headers)

    if response.status_code == 200 and response.text.strip():
        try:
            data = response.json()
            current_category = None

            for player in data.get('player', []):
                if 'id' not in player:
                    # This is a category header
                    current_category = player.get('name')
                    continue

                cursor.execute("""
                    INSERT INTO team_players_updated (
                        player_id, team_id, player_name, batting_style, bowling_style, image_id, category
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        player_name = VALUES(player_name),
                        batting_style = VALUES(batting_style),
                        bowling_style = VALUES(bowling_style),
                        image_id = VALUES(image_id),
                        category = VALUES(category)
                """, (
                    int(player['id']),
                    team_id,
                    player.get('name'),
                    player.get('battingStyle'),
                    player.get('bowlingStyle'),
                    player.get('imageId'),
                    current_category
                ))
            conn.commit()
            print(f"✅ Player data inserted for team {team_id}")
        except ValueError:
            print(f"⚠️ JSON decode failed for team {team_id}")
    else:
        print(f"❌ Failed to fetch players for team {team_id}. Status code: {response.status_code}")

    time.sleep(2)
cursor.close()
conn.close()
print("✅ All team player data with categories inserted successfully.")    

In [5]:
import requests
import pandas as pd
import mysql.connector
import time

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)
cursor = conn.cursor()

import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"

querystring = {"year":"2024"}

headers = {
	"x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

series_2024 = response.json()

In [6]:
print(series_2024)

{'seriesMapProto': [{'date': '2024', 'series': [{'id': 8553, 'name': 'Sri Lanka tour of New Zealand, 2024-25', 'startDt': '1734912000000', 'endDt': '1736553600000'}, {'id': 9297, 'name': 'Gulf Cricket T20I Championship, 2024', 'startDt': '1734048000000', 'endDt': '1734739200000'}, {'id': 8044, 'name': 'Pakistan tour of South Africa, 2024 -25', 'startDt': '1733788800000', 'endDt': '1736208000000'}, {'id': 9171, 'name': 'Afghanistan tour of Zimbabwe, 2024-25', 'startDt': '1733702400000', 'endDt': '1736121600000'}, {'id': 9265, 'name': 'ICC Mens T20 World Cup Americas Sub Regional Qualifier, 2024', 'startDt': '1733443200000', 'endDt': '1734307200000'}, {'id': 9289, 'name': 'Africa Continental Cup 2024', 'startDt': '1733270400000', 'endDt': '1734134400000'}, {'id': 8058, 'name': 'Sri Lanka tour of South Africa, 2024', 'startDt': '1732665600000', 'endDt': '1733702400000'}, {'id': 8494, 'name': 'Pakistan tour of Zimbabwe, 2024', 'startDt': '1732406400000', 'endDt': '1733356800000', 'isFantas

In [7]:
# Extract series list from series_2024
series_list = series_2024.get('seriesMapProto', [])

# Convert to DataFrame
df_series = pd.DataFrame(series_list)

# Print the DataFrame
print(df_series)

   date                                             series
0  2024  [{'id': 8553, 'name': 'Sri Lanka tour of New Z...


In [11]:
import requests
import pandas as pd
import mysql.connector
from datetime import datetime

# Step 1: Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)
cursor = conn.cursor()

# Step 2: Fetch series data from API
url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"
querystring = {"year": "2024"}
headers = {
    "x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)
series_2024 = response.json()

# Step 3: Extract and transform series info
series_info = []
for item in series_2024.get('seriesMapProto', []):
    for s in item.get('series', []):
        series_info.append({
            'id': s.get('id'),
            'name': s.get('name'),
            'startDt': datetime.fromtimestamp(int(s.get('startDt')) / 1000).strftime('%Y-%m-%d'),
            'endDt': datetime.fromtimestamp(int(s.get('endDt')) / 1000).strftime('%Y-%m-%d')
        })

df_series_info = pd.DataFrame(series_info)

# Step 4: Create MySQL table and insert data
cursor.execute("""
    CREATE TABLE IF NOT EXISTS series_2024_info (
        series_id INT PRIMARY KEY,
        name VARCHAR(255),
        startDt DATE,
        endDt DATE
    )
""")

for _, row in df_series_info.iterrows():
    cursor.execute("""
        INSERT INTO series_2024_info (series_id, name, startDt, endDt)
        VALUES (%s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            name = VALUES(name),
            startDt = VALUES(startDt),
            endDt = VALUES(endDt)
    """, (row['id'], row['name'], row['startDt'], row['endDt']))

conn.commit()
cursor.close()
conn.close()

print("\n✅ Series data for 2024 successfully stored in MySQL table 'series_2024_info'.")


✅ Series data for 2024 successfully stored in MySQL table 'series_2024_info'.


In [ ]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/40381/scard"
#f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/scard"

headers = {
	"x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)


data_scorecard = response.json()

In [ ]:
# matchwise score card with more details 
# dont modify this core 
import mysql.connector
import requests
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)

cursor = conn.cursor()
cursor.execute("SELECT match_id FROM match_details where match_id = 130146")
match_ids = [row[0] for row in cursor.fetchall()]

headers = {
    "x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

batsman_list = []
bowling_list = []
fow_list = []
extras_list = []
powerplay_list = []
partnership_list = []

for match_id in match_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/scard"
    try:
        response = requests.get(url, headers=headers)
        data_scorecard = response.json()

        for innings in data_scorecard.get("scorecard", []):
            innings_id = innings.get("inningsid")
            bat_team = innings.get("batteamname")

            # Batsmen
            for batsman in innings.get("batsman", []):
                batsman_list.append({
                    "match_id": match_id,
                    "inningsid": innings_id,
                    "batting_teamname": bat_team,
                    "Player Name": batsman.get("name"),
                    "player Id": batsman.get("id"),
                    "Runs": batsman.get("runs"),
                    "Balls": batsman.get("balls"),
                    "Fours": batsman.get("fours"),
                    "Sixes": batsman.get("sixes"),
                    "Strike Rate": batsman.get("strkrate"),
                    "Out Description": batsman.get("outdec"),
                    "Is Captain": batsman.get("iscaptain"),
                    "Is Keeper": batsman.get("iskeeper"),
                    "Is Overseas": batsman.get("isoverseas")
                })

            # Bowlers
            for bowler in innings.get("bowler", []):
                bowling_list.append({
                    "match_id": match_id,
                    "inningsid": innings_id,
                    "Player Name": bowler.get("name"),
                    "player Id": bowler.get("id"),
                    "Overs": bowler.get("overs"),
                    "Maiden": bowler.get("maidens"),
                    "Wickets": bowler.get("wickets"),
                    "Runs": bowler.get("runs"),
                    "Economy": bowler.get("economy"),
                    "Is Captain": bowler.get("iscaptain"),
                    "Is Keeper": bowler.get("iskeeper")
                })

            # Fall of Wickets
            for fow in innings.get("fow", {}).get("fow", []):
                fow_list.append({
                    "match_id": match_id,
                    "inningsid": innings_id,
                    "Batsman Name": fow.get("batsmanname"),
                    "Batsman ID": fow.get("batsmanid"),
                    "Over": fow.get("overnbr"),
                    "Team Runs": fow.get("runs"),
                    "Ball Number": fow.get("ballnbr")
                })

            # Extras
            extras = innings.get("extras", {})
            extras_list.append({
                "match_id": match_id,
                "inningsid": innings_id,
                "Leg Byes": extras.get("legbyes"),
                "Byes": extras.get("byes"),
                "Wides": extras.get("wides"),
                "No Balls": extras.get("noballs"),
                "Penalty": extras.get("penalty"),
                "Total Extras": extras.get("total")
            })

            # Powerplay
            for pp in innings.get("pp", {}).get("powerplay", []):
                powerplay_list.append({
                    "match_id": match_id,
                    "inningsid": innings_id,
                    "Type": pp.get("pptype"),
                    "From Over": pp.get("ovrfrom"),
                    "To Over": pp.get("ovrto"),
                    "Runs": pp.get("run"),
                    "Wickets": pp.get("wickets")
                })

            # Partnerships
            for p in innings.get("partnership", {}).get("partnership", []):
                partnership_list.append({
                    "match_id": match_id,
                    "inningsid": innings_id,
                    "Player1_id" :p.get("bat1id"),
                    "Bat1 Name": p.get("bat1name"),
                    "Bat1 Runs": p.get("bat1runs"),
                    "Bat1 Fours": p.get("bat1fours"),
                    "Bat1 Sixes": p.get("bat1sixes"),
                    "Bat2 Name": p.get("bat2name"),
                    "Player2_id" :p.get("bat2id"),
                    "Bat2 Runs": p.get("bat2runs"),
                    "Bat2 Fours": p.get("bat2fours"),
                    "Bat2 Sixes": p.get("bat2sixes"),
                    "Total Runs": p.get("totalruns"),
                    "Total Balls": p.get("totalballs")
                })

    except Exception as e:
        print(f"Error fetching match {match_id}: {e}")

# Convert to DataFrames
df_batsmen = pd.DataFrame(batsman_list)
df_bowling = pd.DataFrame(bowling_list)
df_fow = pd.DataFrame(fow_list)
df_extras = pd.DataFrame(extras_list)
df_powerplay = pd.DataFrame(powerplay_list)
df_partnership = pd.DataFrame(partnership_list)

# Display
# print("Batsmen:\n", df_batsmen)
# print("Bowlers:\n", df_bowling)
# print("Fall of Wickets:\n", df_fow)
# print("Extras:\n", df_extras)
# print("Powerplay:\n", df_powerplay)
# print("Partnerships:\n", df_partnership)

In [62]:
import mysql.connector
import requests
import time

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Welcome*1",
    database="cricbuzz"
)
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS scorecard_match_batsmen (
    match_id INT,
    inningsid INT,
    batting_teamname VARCHAR(100),
    player_name VARCHAR(100),
    player_id INT,
    runs INT,
    balls INT,
    fours INT,
    sixes INT,
    strike_rate VARCHAR(10),
    out_description TEXT,
    is_captain BOOLEAN,
    is_keeper BOOLEAN,
    is_overseas BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS scorecard_match_bowling (
    match_id INT,
    inningsid INT,
    player_name VARCHAR(100),
    player_id INT,
    overs VARCHAR(10),
    maiden INT,
    wickets INT,
    runs INT,
    economy VARCHAR(10),
    is_captain BOOLEAN,
    is_keeper BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS scorecard_match_partnership (
    match_id INT,
    inningsid INT,
    player1_id INT,
    bat1_name VARCHAR(100),
    bat1_runs INT,
    bat1_fours INT,
    bat1_sixes INT,
    player2_id INT,
    bat2_name VARCHAR(100),
    bat2_runs INT,
    bat2_fours INT,
    bat2_sixes INT,
    total_runs INT,
    total_balls INT
)
""")

# Fetch match IDs
cursor.execute("SELECT match_id FROM match_details")
match_ids = [row[0] for row in cursor.fetchall()]

# API headers
headers = {
    "x-rapidapi-key": "af06ca3954mshac617103945081ap11717ejsn97324ff1b680",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# Loop through each match
for match_id in match_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/scard"
    try:
        response = requests.get(url, headers=headers)
        data_scorecard = response.json()

        for innings in data_scorecard.get("scorecard", []):
            innings_id = innings.get("inningsid")
            bat_team = innings.get("batteamname")

            # Batsmen
            for batsman in innings.get("batsman", []):
                cursor.execute("""
                    INSERT INTO scorecard_match_batsmen (
                        match_id, inningsid, batting_teamname, player_name, player_id,
                        runs, balls, fours, sixes, strike_rate, out_description,
                        is_captain, is_keeper, is_overseas
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    match_id, innings_id, bat_team, batsman.get("name"), batsman.get("id"),
                    batsman.get("runs"), batsman.get("balls"), batsman.get("fours"), batsman.get("sixes"),
                    batsman.get("strkrate"), batsman.get("outdec"),
                    batsman.get("iscaptain"), batsman.get("iskeeper"), batsman.get("isoverseas")
                ))

            # Bowlers
            for bowler in innings.get("bowler", []):
                cursor.execute("""
                    INSERT INTO scorecard_match_bowling (
                        match_id, inningsid, player_name, player_id, overs, maiden,
                        wickets, runs, economy, is_captain, is_keeper
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    match_id, innings_id, bowler.get("name"), bowler.get("id"), bowler.get("overs"),
                    bowler.get("maidens"), bowler.get("wickets"), bowler.get("runs"),
                    bowler.get("economy"), bowler.get("iscaptain"), bowler.get("iskeeper")
                ))

            # Partnerships
            for p in innings.get("partnership", {}).get("partnership", []):
                cursor.execute("""
                    INSERT INTO scorecard_match_partnership (
                        match_id, inningsid, player1_id, bat1_name, bat1_runs, bat1_fours,
                        bat1_sixes, player2_id, bat2_name, bat2_runs, bat2_fours,
                        bat2_sixes, total_runs, total_balls
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    match_id, innings_id, p.get("bat1id"), p.get("bat1name"), p.get("bat1runs"),
                    p.get("bat1fours"), p.get("bat1sixes"), p.get("bat2id"), p.get("bat2name"),
                    p.get("bat2runs"), p.get("bat2fours"), p.get("bat2sixes"),
                    p.get("totalruns"), p.get("totalballs")
                ))

        conn.commit()
        print(f"✅ Match {match_id} data inserted successfully.")
        time.sleep(0.5)  # Optional: pause to avoid API throttling

    except Exception as e:
        print(f"❌ Error fetching match {match_id}: {e}")
        with open("error_log.txt", "a") as log:
            log.write(f"Match {match_id} failed: {e}\n")

✅ Match 89241 data inserted successfully.
✅ Match 89248 data inserted successfully.
✅ Match 89252 data inserted successfully.
✅ Match 89259 data inserted successfully.
✅ Match 89262 data inserted successfully.
✅ Match 89269 data inserted successfully.
✅ Match 89273 data inserted successfully.
✅ Match 89276 data inserted successfully.
✅ Match 89280 data inserted successfully.
✅ Match 89287 data inserted successfully.
✅ Match 89290 data inserted successfully.
✅ Match 89294 data inserted successfully.
✅ Match 94278 data inserted successfully.
✅ Match 94287 data inserted successfully.
✅ Match 94296 data inserted successfully.
✅ Match 94305 data inserted successfully.
✅ Match 94314 data inserted successfully.
✅ Match 94318 data inserted successfully.
✅ Match 103287 data inserted successfully.
✅ Match 103294 data inserted successfully.
✅ Match 103296 data inserted successfully.
✅ Match 103305 data inserted successfully.
✅ Match 103314 data inserted successfully.
✅ Match 103323 data inserted 